https://www.kaggle.com/code/khuonglephuc/lesionsegmentmedsam/notebook

https://www.kaggle.com/datasets/tienle65/isicsegmentupdate

In [1]:
# Install necessary packages
!pip install monai
!pip install segment-anything
!pip install git+https://github.com/bowang-lab/MedSAM.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 5.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-10.3.9

In [2]:
import os
import random
import shutil
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm

# Define paths
train_img_path = "/kaggle/input/isicsegmentupdate/merged_train/images"
train_mask_path = "/kaggle/input/isicsegmentupdate/merged_train/masks"

val_img_path = "/kaggle/input/isicsegmentupdate/merged_val/images"
val_mask_path = "/kaggle/input/isicsegmentupdate/merged_val/masks"


In [3]:
import cv2
import torch
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from segment_anything.utils.transforms import ResizeLongestSide


class ISICDataset(Dataset):
    def __init__(self, img_dir, mask_dir, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transform = transform
        
        # Tìm tất cả file ảnh
        self.images = sorted(glob(os.path.join(img_dir, "*.jpg")))
        if len(self.images) == 0:
            # Thử tìm kiếm với các đuôi khác
            self.images = sorted(glob(os.path.join(img_dir, "*.JPG")))
            if len(self.images) == 0:
                self.images = sorted(glob(os.path.join(img_dir, "*.jpeg")))
        
        print(f"Tìm thấy {len(self.images)} ảnh trong {img_dir}")
        
        # Tìm mask tương ứng
        self.masks = []
        self.valid_indices = []
        
        for i, img_path in enumerate(self.images):
            img_id = os.path.basename(img_path).split('.')[0]
            mask_path = os.path.join(mask_dir, f"{img_id}_segmentation.png")
            
            if os.path.exists(mask_path):
                self.masks.append(mask_path)
                self.valid_indices.append(i)
            else:
                print(f"Warning: Không tìm thấy mask cho {img_id}")
        
        # Chỉ giữ lại những ảnh có mask
        self.images = [self.images[i] for i in self.valid_indices]
        
        print(f"Số lượng cặp ảnh-mask hợp lệ: {len(self.images)}")
        
        if len(self.images) == 0:
            raise ValueError(f"Không tìm thấy dữ liệu nào trong {img_dir} và {mask_dir}")
        
        # Khởi tạo SAM transform
        self.sam_transform = ResizeLongestSide(1024)  # Resize theo yêu cầu của SAM ViT-H
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        mask_path = self.masks[idx]
        
        # Đọc ảnh và mask
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = (mask > 0).astype(np.float32)  # Chuyển thành binary mask
        
        # Áp dụng transform thông thường (nếu có)
        if self.transform:
            transformed = self.transform(image=image, mask=mask)
            image = transformed["image"]
            mask = transformed["mask"]
        
        # Tìm bbox từ mask
        y_indices, x_indices = np.where(mask > 0)
        if len(y_indices) > 0 and len(x_indices) > 0:
            x_min, x_max = np.min(x_indices), np.max(x_indices)
            y_min, y_max = np.min(y_indices), np.max(y_indices)
            bbox = np.array([x_min, y_min, x_max, y_max])
        else:
            # Nếu mask trống, dùng toàn bộ ảnh
            bbox = np.array([0, 0, image.shape[1]-1, image.shape[0]-1])
        
        # Lấy điểm ngẫu nhiên trong mask để làm prompt
        if len(y_indices) > 0:
            point_idx = np.random.randint(0, len(y_indices))
            point = np.array([x_indices[point_idx], y_indices[point_idx]])
        else:
            # Nếu mask trống, dùng điểm giữa ảnh
            point = np.array([image.shape[1] // 2, image.shape[0] // 2])
        
        # Áp dụng SAM transform
        # Chuyển đổi ảnh thành tensor trước khi áp dụng SAM transform
        image_tensor = torch.from_numpy(image).permute(2, 0, 1)
        
        # Áp dụng SAM transform để resize ảnh theo đúng yêu cầu của SAM
        image_1024 = self.sam_transform.apply_image(image)
        image_1024_tensor = torch.from_numpy(image_1024).permute(2, 0, 1).float() / 255.0
        
        # Chuyển đổi điểm và bbox theo tỷ lệ mới
        input_size = image.shape[:2]
        new_size = image_1024.shape[:2]
        
        # Scale điểm và bbox theo kích thước mới
        point_1024 = self.sam_transform.apply_coords(point.reshape(1, 2), input_size).flatten()
        bbox_1024 = self.sam_transform.apply_coords(
            np.array([[bbox[0], bbox[1]], [bbox[2], bbox[3]]]), input_size
        ).flatten()
        
        mask_tensor = torch.from_numpy(mask).unsqueeze(0)
        point_tensor = torch.from_numpy(point_1024)
        bbox_tensor = torch.from_numpy(bbox_1024)
        
        return {
            "image": image_1024_tensor,
            "mask": mask_tensor,
            "bbox": bbox_tensor,
            "point": point_tensor,
            "image_path": img_path
        }

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.amp import autocast
from torch.amp import GradScaler
import albumentations as A
from albumentations.pytorch import ToTensorV2
from segment_anything import sam_model_registry
from segment_anything.utils.transforms import ResizeLongestSide
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import os

# Download the pre-trained SAM model (ViT-H is the most powerful)
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth

# Set up transforms
train_transform = A.Compose([
    A.Resize(256, 256),  # Giữ nguyên hoặc giảm xuống nếu cần
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
])

val_transform = A.Compose([
    A.Resize(256, 256),
])

# Create datasets
train_dataset = ISICDataset(
    img_dir=train_img_path,  # Sử dụng đường dẫn gốc, không xử lý lại
    mask_dir=train_mask_path,  # Sử dụng đường dẫn gốc, không xử lý lại
    transform=train_transform
)

val_dataset = ISICDataset(
    img_dir=val_img_path,
    mask_dir=val_mask_path,
    transform=val_transform
)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=2)

# Initialize the SAM model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_type = "vit_b"  # Sử dụng vit_b thay vì vit_h
sam_checkpoint = "sam_vit_b_01ec64.pth"  # Checkpoint tương ứng

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device)

# Fine-tune only the image encoder part of SAM
for name, param in sam.named_parameters():
    param.requires_grad = False  # Freeze tất cả trước
    
# Chỉ unfreeze mask decoder để finetune
for name, param in sam.mask_decoder.named_parameters():
    param.requires_grad = True

# Optimizer
optimizer = optim.Adam([p for p in sam.parameters() if p.requires_grad], lr=1e-5)
loss_fn = nn.BCEWithLogitsLoss()

# MedSAM training loop
def train_medsam(model, train_loader, val_loader, optimizer, loss_fn, num_epochs=10):
    sam_transform = ResizeLongestSide(model.image_encoder.img_size)
    best_val_loss = float('inf')
    scaler = GradScaler()  # Thêm gradient scaler cho mixed precision
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
            images = batch["image"].to(device)
            masks = batch["mask"].to(device)
            points = batch["point"].to(device).unsqueeze(1)
            point_labels = torch.ones(points.shape[0], points.shape[1], dtype=torch.int, device=device)
            
            # Sử dụng autocast cho mixed precision
            with autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
                # Get image embeddings
                with torch.no_grad():  # Đảm bảo không tính gradient cho encoder
                    image_embeddings = model.image_encoder(images)
                
                # Get sparse embeddings with point prompts
                sparse_embeddings, dense_embeddings = model.prompt_encoder(
                    points=(points, point_labels),
                    boxes=None,
                    masks=None,
                )
                                            
                # Predict masks
                mask_predictions, _ = model.mask_decoder(
                    image_embeddings=image_embeddings,
                    image_pe=model.prompt_encoder.get_dense_pe(),
                    sparse_prompt_embeddings=sparse_embeddings,
                    dense_prompt_embeddings=dense_embeddings,
                    multimask_output=False,
                )
                
                # Calculate loss
                loss = loss_fn(mask_predictions[:, 0], masks.squeeze(1))
            
            # Sử dụng gradient scaler cho mixed precision
            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        
        # Validation
        model.eval()
        val_loss = 0
        iou_scores = []
        
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
                images = batch["image"].to(device)
                masks = batch["mask"].to(device)
                points = batch["point"].to(device).unsqueeze(1)
                point_labels = torch.ones(points.shape[0], points.shape[1], dtype=torch.int, device=device)
                
                # Sử dụng autocast cho mixed precision
                with autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
                    # Get image embeddings
                    with torch.no_grad():  # Đảm bảo không tính gradient cho encoder
                        image_embeddings = model.image_encoder(images)
                    
                    # Get sparse embeddings with point prompts
                    sparse_embeddings, dense_embeddings = model.prompt_encoder(
                        points=(points, point_labels),
                        boxes=None,
                        masks=None,
                    )                                        
                    mask_predictions, _ = model.mask_decoder(
                        image_embeddings=image_embeddings,
                        image_pe=model.prompt_encoder.get_dense_pe(),
                        sparse_prompt_embeddings=sparse_embeddings,
                        dense_prompt_embeddings=dense_embeddings,
                        multimask_output=False,
                    )
                    
                    # Calculate loss and IoU
                    loss = loss_fn(mask_predictions[:, 0], masks.squeeze(1))
                
                val_loss += loss.item()
                
                # Calculate IoU
                pred_masks = (mask_predictions[:, 0] > 0).float()
                intersection = (pred_masks * masks.squeeze(1)).sum((1, 2))
                union = pred_masks.sum((1, 2)) + masks.squeeze(1).sum((1, 2)) - intersection
                batch_iou = (intersection / (union + 1e-6)).mean().item()
                iou_scores.append(batch_iou)
        
        val_loss /= len(val_loader)
        mean_iou = np.mean(iou_scores)
        
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Mean IoU: {mean_iou:.4f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "best_medsam_model.pth")
            print(f"Saved best model with val loss: {best_val_loss:.4f}")
    
    return model

#Train MedSAM
trained_model = train_medsam(sam, train_loader, val_loader, optimizer, loss_fn, num_epochs=40)

--2025-04-20 10:27:32--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.171.22.68, 3.171.22.118, 3.171.22.33, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.171.22.68|:443... connected.
HTTP request sent, awaiting response... 

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


200 OK
Length: 375042383 (358M) [binary/octet-stream]
Saving to: ‘sam_vit_b_01ec64.pth’

sam_vit_b_01ec64.pt 100%[===================>] 357.67M   275MB/s    in 1.3s    

2025-04-20 10:27:33 (275 MB/s) - ‘sam_vit_b_01ec64.pth’ saved [375042383/375042383]

Tìm thấy 3444 ảnh trong /kaggle/input/isicsegmentupdate/merged_train/images
Số lượng cặp ảnh-mask hợp lệ: 3444
Tìm thấy 250 ảnh trong /kaggle/input/isicsegmentupdate/merged_val/images
Số lượng cặp ảnh-mask hợp lệ: 250


/usr/local/lib/python3.11/dist-packages/segment_anything/build_sam.py:144: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f, map_location=torch.device

Epoch 1/40, Train Loss: 0.2357, Val Loss: 0.2220, Mean IoU: 0.7402
Saved best model with val loss: 0.2220


...

Epoch 38/40, Train Loss: 0.1317, Val Loss: 0.1555, Mean IoU: 0.8033


Epoch 39/40 - Validation: 100%|██████████| 125/125 [01:04<00:00,  1.95it/s]


Epoch 39/40, Train Loss: 0.1297, Val Loss: 0.1571, Mean IoU: 0.8000


Epoch 40/40 - Validation: 100%|██████████| 125/125 [01:04<00:00,  1.93it/s]

Epoch 40/40, Train Loss: 0.1302, Val Loss: 0.1503, Mean IoU: 0.8107
